# MLflow integration with abrege project

### MLflow experiment setup

In [1]:
import mlflow
import os

tracking_uri = os.environ["MLFLOW_TRACKING_URI"]
mlflow.set_tracking_uri(tracking_uri)

In [17]:
def get_or_create_experiment(name: str):
    if (experiment := mlflow.get_experiment_by_name(name)) is not None:
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(name)

experiment_id = get_or_create_experiment("abrege")
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1716541753649, experiment_id='4', last_update_time=1716541753649, lifecycle_stage='active', name='abrege', tags={}>

### Data preparation

Le corpus de test n'est composé que de quelques gros textes car la méthode selfcheck rends très long l'évaluation de métrique (en plus du résumé)

In [18]:
from datasets import load_dataset

wiki = load_dataset("wikipedia", language="fr", date="20220301", trust_remote_code=True)

In [19]:
content = ("https://fr.wikipedia.org/wiki/Albert%20Camus", 
           "https://fr.wikipedia.org/wiki/George%20Orwell", 
           "https://fr.wikipedia.org/wiki/Jules%20Verne", 
           "https://fr.wikipedia.org/wiki/Victor%20Hugo", 
           "https://fr.wikipedia.org/wiki/Ludwig%20van%20Beethoven")

idx_list = [wiki["train"]["url"].index(url) for url in content]

dataset = wiki["train"].select(idx_list)

### Data Logging

On enregistre les données dans une run, qui sera notre run parente pour plus part (bien retenir l'id)

In [20]:
run_name = "demo"

with mlflow.start_run(run_name=run_name) as current_run:
    mlflow.log_input(mlflow.data.huggingface_dataset.from_huggingface(dataset), context="validation")
    run_id = current_run.info.run_id

### Hyperparameters definition

On définit dans cette section les différents hyperparamètres qu'on va exploerer de manière catégorique.
On définit aussi le llm qui va servir à l'évaluation

#### LLM initialization

In [2]:
from langchain_openai import ChatOpenAI
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from abrege.summary_chain import EmbeddingModel

api_key = os.environ["OPENAI_API_KEY"]
base_url = os.environ["OPENAI_API_BASE"]
embeddings_base_url = os.environ["OPENAI_EMBEDDING_API_BASE"]

llm = ChatOpenAI(
    api_key=api_key,
    base_url=base_url,
    model="vicuna",
    temperature=0
)

embeddings_function = OpenAIEmbeddingFunction(
    api_key=api_key,
    api_base=embeddings_base_url
)
embedding_model = EmbeddingModel(embeddings_function)

llm_params = {
    "model": "vicuna",
    "temperature": 0,
    "embedding_model": "solon"
}

ModuleNotFoundError: No module named 'abrege'

#### Parameters definition

In [22]:
categorical_param = {
    "method": ["map_reduce", "k-means", "text_rank"],
    "context_size": [2500, 5000, 10_000]
}

fixed_params = {
    "language": "French",
    "size": 200,
}

#### MLflow run tagging

In [23]:
with mlflow.start_run(run_id=run_id):
    mlflow.set_tags({
        "fixed_params": fixed_params,
        "evaluated_params": categorical_param
    })

### Objective Function

In [24]:
import optuna

# override Optuna's default logging to ERROR only
optuna.logging.set_verbosity(optuna.logging.ERROR)

def champion_callback(study, frozen_trial):
    """
    Logging callback that will report when a new trial iteration
    improves upon existing best trial values

    Note: not data race safe !
    """

    winner = study.user_attrs.get("winner", None)

    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value)
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial tiral {frozen_trial.number} achieved value {frozen_trial.value}")

In [25]:
from abrege import summarize_chain_builder
from abrege.selfcheck import selfcheck
from langchain_core.documents import Document
import time

def objective(trial):
    with mlflow.start_run(nested=True):
        # Define hyperparameters
        params = fixed_params.copy()
        for param, value_list in categorical_param.items():
            params[param] = trial.suggest_categorical(param, value_list)

        mlflow.log_params(params)
        # Evaluate the summary
        tot_time = 0
        tot_score = 0
        chain = summarize_chain_builder(
            llm=llm,
            embedding_model=embedding_model,
            **params
        )
        for page in dataset:
            # Make the summary
            text = page["text"]
            deb = time.perf_counter()
            summary = chain.invoke(text)
            tot_time += time.perf_counter() - deb

            # Evaluate the summary
            documents = [Document(page_content=text)]
            tot_score += selfcheck(llm=llm, docs=documents, summarize_to_eval=summary)

        mlflow.log_metric("selfcheck_score", tot_score / len(dataset))
        mlflow.log_metric("avg_time", tot_time / len(dataset))

    return tot_score / len(dataset)

### Hyperparameter tuning

In [26]:
with mlflow.start_run(run_id=run_id, nested=True) as parent_run:

    study = optuna.create_study(direction="maximize")

    study.optimize(objective, n_trials=10, callbacks=[champion_callback])

    best_param = study.best_params | fixed_params

    mlflow.log_params(best_param)


/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
                the prompt : response="번역결과  \n질문: 아래 내용에서 주어진 문장이 사실인지 여

Initial tiral 0 achieved value 0.8073934837092732


/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingf

Trial 2 achieved value: 0.8312030075187969 with  0.0286% improvement


/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jerome/miniforge3/envs/abrege/lib/python3.11/site-packages/huggingf